<a href="https://colab.research.google.com/github/ahcamachod/1878-analisis-y-clasificacion-de-rostros-vision-por-computador-con-opencv/blob/main/opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis y Clasificación de Rostros: Visión por Computador con OpenCV

En este notebook estaremos desarrollando el código para trabajar con análisis facial utilizando Open Computer Vision.

Emplearemos algunas imágenes para realizar los ejercicios, y particularmente, tomaremos como referencia una base de imágenes de Georgia Tech (*http://www.anefian.com/research/gt_db.zip*).

La documentación referente a los principales paquetes que estaremos usando se relaciona a continuación:

* *https://docs.opencv.org/4.x/*
* *http://dlib.net/python/*
* *https://pillow.readthedocs.io/en/stable/*

## 1. Conociendo el problema

## 2. Segmentación de rostros

## 3. Clasificación de rostros

## 4. Algoritmos de clasificación

## 5. Marcos faciales

## 6. Aplicaciones del análisis facial